<a href="https://colab.research.google.com/github/FairuzAthallah88/Machine-Learning-Models/blob/main/LOAD_MODEL_MobileNetV3_XPLORIN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UJI MODEL CNN XPLORIN MobileNetV3
MobileNetV3


1. Inisialisasi dan Muat Model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
from google.colab import files, drive
import os
import io

# MobileNetV3
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model

# --- PATH ---
drive.mount('/content/drive')
MODEL_PATH = '/content/drive/MyDrive/DATASET_CNN_XPLORIN/MODEL_ASSETS/best_mobilenet_v3_large.h5'

IMG_HEIGHT = 224
IMG_WIDTH = 224
THRESHOLD_SCORE = 0.85

# Daftar Nama Kelas
CLASS_NAMES = [
    'Celimpungan', 'Es Kacang Merah', 'Kemplang', 'Kue Delapan Jam',
    'Kue Gandus', 'Laksan', 'Martabak Kari', 'Mie Celor',
    'Model', 'Pempek Adaan', 'Pempek Kapal Selam', 'Pempek Keriting',
    'Pempek Kulit', 'Pempek Lenggang', 'Pempek Lenjer', 'Pindang Patin',
    'Tekwan'
]

# --- MUAT MODEL ---
print(f"--- Memuat Model MobileNetV3 dari: {MODEL_PATH} ---")

try:
    # 1. Bangun Base Model
    base_model = MobileNetV3Large(
        weights=None,
        include_top=False,
        input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)
    )

    # 2. Bangun Head
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.6)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.6)(x)
    predictions = Dense(len(CLASS_NAMES), activation='softmax')(x)

    # 3. Gabungkan
    global model
    model = Model(inputs=base_model.input, outputs=predictions)

    # 4. Load Bobot
    model.load_weights(MODEL_PATH)

    print("✅ Siap untuk prediksi.")
except Exception as e:
    print(f"❌ Gagal memuat model. Error: {e}")

2. Definisi Fungsi Prediksi

In [ ]:
def classify_uploaded_image(uploaded_data, uploaded_filename):
    """
    Memproses gambar dengan standar MobileNetV3 dan menerapkan ambang batas 65%.
    """
    try:
        # 1. Buka Gambar (Standar Preprocessing)
        img = Image.open(io.BytesIO(uploaded_data)).convert('RGB')
        img = img.resize((IMG_WIDTH, IMG_HEIGHT))

        # 2. Preprocessing
        img_array = np.array(img, dtype=np.float32)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        # 3. Prediksi
        print("\n--- Memulai Prediksi ---")
        predictions = model.predict(img_array)

        # 4. Ekstraksi Hasil
        predicted_index = np.argmax(predictions[0])
        confidence_score = predictions[0][predicted_index] * 100
        predicted_class = CLASS_NAMES[predicted_index]

        # 5. LOGIKA THRESHOLD
        if confidence_score / 100 < THRESHOLD_SCORE:
            display_text = "TIDAK DIKENALI"
            title_color = 'blue'
            detail_message = f"(Keyakinan < {THRESHOLD_SCORE*100:.0f}%)"
            user_guidance = "SARAN: Coba ambil gambar objek lebih jelas, fokus, dan tidak terpotong."
        else:
            # Model yakin dan hasilnya valid
            display_text = f"PREDIKSI: {predicted_class}"
            title_color = 'red'
            detail_message = f"KEYAKINAN: {confidence_score:.2f}%"
            user_guidance = ""

        # 6. Menampilkan Gambar dan Hasil
        plt.figure(figsize=(8, 8))
        plt.imshow(img)

        # Gabungkan detail_message dan user_guidance di judul plot
        plot_title = f"{display_text}\n{detail_message}"
        if user_guidance:
             plot_title += f"\n\n{user_guidance}"

        plt.title(plot_title, fontsize=14, color=title_color, fontweight='bold')
        plt.axis('off')
        plt.show()

        print("\n===========================================")
        print(f" HASIL AKHIR: {display_text.replace('PREDIKSI: ', '')}")
        print(f" {detail_message}")

        if user_guidance:
            print(f" SARAN: {user_guidance.replace('SARAN: ', '')}")

        print("===========================================")

    except Exception as e:
        print(f"❌ Terjadi error saat pemrosesan atau prediksi: {e}")

print("✅ Fungsi prediksi MobileNetV3 telah didefinisikan dengan panduan user.")

3. Uji Gambar

In [ ]:
# --- UJI GAMBAR ---
print("--- UPLOAD MAKANAN ---")
print("!!PASTIKAN GAMBAR JELAS!!")
uploaded = files.upload()

if uploaded:
    uploaded_filename = list(uploaded.keys())[0]
    uploaded_data = uploaded[uploaded_filename]
    print(f"File '{uploaded_filename}' berhasil diupload. Mengklasifikasi...")

    # Panggil fungsi prediksi
    classify_uploaded_image(uploaded_data, uploaded_filename)
else:
    print("❌ Tidak ada file yang diupload.")